In [1]:
import torch
import gym
import numpy as np

from ji_dog_net import PPO
from ji_dog_net import ActorCritic
from ji_dog_net import process_state

## Check device

In [2]:
print("============================================================================================")
# set device to cpu or cuda
device = torch.device('cpu')
if(torch.cuda.is_available()): 
    device = torch.device('cuda:0') 
    torch.cuda.empty_cache()
    print("Device set to : " + str(torch.cuda.get_device_name(device)))
else:
    print("Device set to : cpu")
print("============================================================================================")


Device set to : NVIDIA GeForce RTX 4060 Laptop GPU


## Test ACNet

In [3]:
from torchviz import make_dot
state_dim = 20  
action_dim = 4 
ppo = PPO(state_dim, action_dim, lr_actor=0.0003, lr_critic=0.001, gamma=0.99, K_epochs=4, eps_clip=0.2)
state = torch.rand((state_dim,)).to(device)  
action, action_logprob, hidden_actor = ppo.policy.act(state)

print("Action output:", action)
print("Action log probability:", action_logprob)

state_value = ppo.policy.evaluate_critic(state)

print("State value output:", state_value)

ppo.buffer.rewards.append(1.0)
ppo.buffer.is_terminals.append(False)


Action output: tensor([[-0.0189, -0.7116, -1.0118, -0.3363]], device='cuda:0')
Action log probability: tensor([-4.1135], device='cuda:0')
State value output: tensor([-0.0809], device='cuda:0', grad_fn=<ViewBackward0>)


## Train PPO

In [4]:
# Environment and simulation setup
from isaacsim import SimulationApp
simulation_app = SimulationApp({"headless": True})

from ji_dog_env_create import Ji_Dog_Env
from tqdm import tqdm  
import numpy as np

if __name__ == "__main__":
    env = Ji_Dog_Env()
    state_dim = 20
    action_dim = 4
    max_training_timesteps = 500
    max_timesteps = 100
    ppo = PPO(state_dim, action_dim, lr_actor=0.0003, lr_critic=0.001, gamma=0.99, K_epochs=4, eps_clip=0.2)

    for episode in tqdm(range(max_training_timesteps), desc="Training Episodes"):
        state = env.reset()
        state = process_state(state)
        state = np.array(state)

        with tqdm(total=max_timesteps, desc=f"Episode {episode}", leave=False) as pbar:
            for t in range(max_timesteps + 1):
                action = ppo.select_action(state)
                state, reward, done, _ = env.step(action[0])
                state = process_state(state)
                state = np.array(state)
                ppo.buffer.rewards.append(reward)
                ppo.buffer.is_terminals.append(done)

                pbar.update(1)

                if done:
                    break
                    
        ppo.update()
        save_path = "Model_Checkpoints/Ji_dog_{}_Episode.pth".format(episode)
        print("save checkpoint path : " + save_path)
        ppo.save(save_path)
        
        if episode % 10 == 0:
            print(f"Episode {episode} completed.")
print('Finished!')

Starting kit application with the following args:  ['/home/bai/.local/share/ov/pkg/isaac-sim-4.2.0/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/home/bai/.local/share/ov/pkg/isaac-sim-4.2.0/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/home/bai/.local/share/ov/pkg/isaac-sim-4.2.0/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True', '--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--/app/fastShutdown=False', '--ext-folder', '/home/bai/.local/share/ov/pkg/isaac-sim-4.2.0/exts', '--ext-folder', '/home/bai/.local/share/ov/pkg/isaac-sim-4.2.0/apps', '--/physics/cudaDevice=0', '--portable', '--no-window', '--/app/window/hideUi=1']
Passing the following args to the base kit application:  ['-f', '/home/bai/.local/share/jupyter/run

Training Episodes:   0%|                        | 1/500 [00:02<22:47,  2.74s/it]

save checkpoint path : Model_Checkpoints/Ji_dog_0_Episode.pth
Episode 0 completed.



Training Episodes:   0%|                        | 2/500 [00:04<18:25,  2.22s/it]

save checkpoint path : Model_Checkpoints/Ji_dog_1_Episode.pth


...

save checkpoint path : Model_Checkpoints/Ji_dog_497_Episode.pth



Training Episodes: 100%|█████████████████████▉| 499/500 [16:02<00:02,  2.05s/it]

save checkpoint path : Model_Checkpoints/Ji_dog_498_Episode.pth



Training Episodes: 100%|██████████████████████| 500/500 [16:04<00:00,  1.93s/it]

save checkpoint path : Model_Checkpoints/Ji_dog_499_Episode.pth
Finished!


In [ ]:
simulation_app.close()